In [77]:
import pymongo
from dotenv import load_dotenv
import os

from utils.preprocess_transcription import remove_ads, identify_host, insert_marker_before_host
from utils.extract_comp_guest import re_extract_comp_guest

from langchain_openai import OpenAIEmbeddings
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_community.chat_models import ChatAnthropic

from langchain.prompts import PromptTemplate
from langchain.output_parsers import PydanticOutputParser

load_dotenv()

True

In [2]:

client = pymongo.MongoClient(os.getenv('mdb_uri'))
DB_NAME = "hibt_prod_db"
COLLECTION_NAME = "hibt_prod_collection"
# ATLAS_VECTOR_SEARCH_INDEX_NAME = "openai_large_index"
MONGODB_COLLECTION = client[DB_NAME][COLLECTION_NAME]
# MONGODB_ANSWER_COLLECTION = client[DB_NAME]["hibt_answer_collection"]

In [3]:
# embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")

# embedded_chunk = embeddings.embed_query(transcript_chunks[0])

In [11]:
org_filenames = MONGODB_COLLECTION.distinct("org_filename")
print(len(org_filenames))


254


In [6]:
from pymongo.collection import Collection

def transcript_pipeline(path: str, filename: str, collection: Collection) -> bool:
    with open(path + filename, 'r') as file:
        transcript = file.read()

    transcript = remove_ads(transcript)
    (host, match) = identify_host(transcript)
    transcript = insert_marker_before_host(transcript, host)
    transcript_chunks = transcript.split('###')
    # embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")
    embeddings = OpenAIEmbeddings(model="text-embedding-3-small")
    meta_data = re_extract_comp_guest(filename)
    meta_data['org_filename'] = filename
    for i, doc in enumerate(transcript_chunks):
        if doc.strip():
            document = meta_data.copy()
            document['transcript'] = doc
            document['embedding'] = embeddings.embed_query(doc)
            document['trans_chunk_num'] = i
            collection.insert_one(document)
    
    return True

In [9]:
# count the number of files in path = '../podscribe_transcription/hibt_main/'
path = '../podscribe_transcription/hibt_main/'
filenames = os.listdir(path)
print(len(filenames))

257


In [8]:
import os
from tqdm import tqdm

successful_files = []
unsuccessful_files = []
path = '../podscribe_transcription/hibt_main/'

# Get the total number of files
total_files = sum(1 for file in os.listdir(path) if file.endswith(".txt"))

with tqdm(total=total_files, desc="Processing files") as pbar:
    for file in os.listdir(path):
        if file.endswith(".txt") and file not in org_filenames:
            try:
                print(f"Processing file: {file}")
                transcript_pipeline(path, file, MONGODB_COLLECTION)
                successful_files.append(file)
            except:
                unsuccessful_files.append(file)
            pbar.update(1)


Processing files:   0%|          | 0/257 [00:00<?, ?it/s]

Processing file: Nov.02 2021, Hear Don Katz on the Amazing Story of Audible’s Founding-transcript.txt
Processing file: Remembering Jake Burton Carpenter-transcript.txt
Processing file: Discord_ Jason Citron-transcript.txt
Processing file: Charging up the electric vehicle market with RJ Scaringe of Rivian-transcript.txt


Processing files:   2%|▏         | 4/257 [00:16<16:59,  4.03s/it]

Processing file: Wayfair_ Niraj Shah & Steve Conine (2018)-transcript.txt


Processing files:   2%|▏         | 5/257 [00:53<53:41, 12.78s/it]

...

Processing files:  64%|██████▍   | 165/257 [1:36:16<49:39, 32.39s/it]

Processing file: Logic_ Logic & Chris Zarou (2018)-transcript.txt


Processing files:  65%|██████▍   | 166/257 [1:36:42<46:15, 30.50s/it]

Processing file: Robinhood_ Vlad Tenev-transcript.txt


Processing files:  65%|██████▍   | 167/257 [1:37:27<52:31, 35.01s/it]


In [12]:
# from tqdm import tqdm
# import concurrent.futures
# path = 'podscribe_transcription/hibt_main/'
# num_threads = 8  

# successful_files = []
# unsuccessful_files = []

# # Get the total number of files
# total_files = sum(1 for file in os.listdir(path) if file.endswith(".txt"))

# with tqdm(total=total_files, desc="Processing files") as pbar:
#     with concurrent.futures.ThreadPoolExecutor(max_workers=num_threads) as executor:
#         for file in os.listdir(path):
#             if file.endswith(".txt"):
#                 print(path, file)
#                 try:
#                     executor.submit(transcript_pipeline, path, file)
#                     successful_files.append(file)
#                 except:
#                     unsuccessful_files.append(file)
#                 pbar.update(1)


### Search for the Answers for each company

In [27]:
client = pymongo.MongoClient(os.getenv('mdb_uri'))
DB_NAME = "hibt_prod_db"
COLLECTION_NAME = "hibt_prod_collection"
ATLAS_VECTOR_SEARCH_INDEX_NAME = "vector_index"
MONGODB_COLLECTION = client[DB_NAME][COLLECTION_NAME]
MONGODB_ANSWER_COLLECTION = client[DB_NAME]["hibt_answer_collection"]

In [103]:
###
hard_luck_prompt = """
    
    You are a transcription analyst assistant. You are given portions of a transcription in order to extract the guests answer.
    We are working with transcriptions from the podcast "How I Built This" where the host Guy Raz interviews entrepreneurs. 
    Typically towards the end of the interview he will ask a question regarding to what extent the entrepreneur attributes their success to skill, hard work, and/or intelligence versus luck.
    Your task is to extract the answer from the transcriptions. Remember that these are transcriptions so words can be mispelled like "lock" instead of "luck" or "intelligents" instead of "intelligence".

    The answer is typically a few sentences long and can be found in the same paragraph or in the following paragraphs depending on their thoughts.
    Here are some examples in <Examples></Examples> XML tags to help you understand the task better.
    <Examples>
    
    Example Question: When you think about the success of what you created, how much do you think it has to do with, with how hard you worked and your intelligence and how much do you think it had to do with luck?
    Example Answer: It's probably 50 50. Look, I, I actually, so I think every CEO who I've met, they have some minimum level of intelligence, right. But they're oftentimes not the smartest person in the room. And I, I would put myself in that category too. Like, I'm, you know, I met some minimum bar, but I'm not the smartest person. What I am is probably quite determined. There is a part of my personality. This might be a little bit like that being on the spectrum thing or whatever, but it's just like, whether it's like a really good day or a really bad day, I'm sort of even keeled. And I'm often just like, okay, well what's the next step? How do we fix it? And let's keep making progress even over like a decade or more. I can be quite resilient in that regard. But I guess the other thing I would say just about luck, absolutely luck played a big role in Coinbase. I mean, there were certain moments early on in our history where if a coin had flipped the other way we might not be here. I think we got very lucky with sort of the timing of when the company decided to launch and when crypto decided to grow, I think that I kind of just kept trying things until eventually one of them hit. Right. Which is, it's a way of unifying those two ideas. It's, it's definitely a lot of luck when it works. But if you just keep trying, you can kind of make your own luck.

    Example Question: Do you, I mean, obviously worked extremely hard making this happen, but I think you did have some incredibly fortunate breaks that you'd like, you couldn't control like male camp. Do you, are, are you a believer in luck or do you think that what happened really has to do with just how strategic and smart you guys were?
    Example Answer: I totally believe in luck. I mean, but I think it comes from constantly striving and looking for that silver bullet. And when I look back on my 20 years, it felt like I was always looking for the answer, always looking for what is the playbook that I'm missing? You know, it's constantly searching. And then all along the way, stumbling upon something that actually does better than my wildest dreams. I don't know. It, it, I, I, I always had in my mind that I was just a bumbling idiot from of a Georgia who stumbled into business. And somewhere out there, someone knew the right answer of how to run my business. Right. And these days, these days, you know, MailChimp is much bigger. And I get invited to ritzy events with some really great business leaders. And I remember being in one recently where a head of one of Microsoft's cloud units, he was, he was speaking about his, how he hosts executive leadership meetings. And he says, we're still winging it. We have no idea. You know, he didn't say it that way, but he's like, we're still experimenting. You know? And this is Microsoft. And they've been around a very, very long time and they know what they're doing. So, you know, it was that moment. I realized, oh my God, nobody has the answer. We're all just winging it. Every one of us is just winging it. I don't know if that's hopeful or horrifying, but that's what I've learned.
    
    Example Question ...do you think it has more to do with just your intelligence and your skill or do you think that luck was a bigger factor in everything that's happened to you
    Example Answer: so the thing I always say about lock I think it is it's the lottery and so you get sort of lottery tickets for how you know it's like how good your idea is how good your team is and how hard you work and so you take those like factors and I think how hard you work in your team or probably even bigger factors than the the idea but you still need to have something that actually would work and so you take those three things together and you get for each sort of our of production or work you get a lottery ticket and so just like the lottery if you unless you own every ticket you're not guaranteed to win because I've seen people that have are smarter than me better than me better idea than me fail because of things like luck like things not going their way and I've seen again the opposite of that and so we have been fortunate to have been have lots of Lucky breaks in our Direction but we also continue to work to get more tickets so that we could win the lottery
    </Examples>

    Transcript to extract answers from:
    <transcript>
    {transcript}
    </transcript>

    If you don't see a relevant question and answer in the transcript, respond with "Question/Answer not found." 
    Only respond with their word-for-word answer. Do not include the question in your response.
    """

In [104]:
from langchain_openai import ChatOpenAI
load_dotenv()

True

In [120]:
import google.generativeai as genai
from google.generativeai.types import HarmCategory, HarmBlockThreshold
GOOGLE_API_KEY=os.getenv('GOOGLE_AISTUDIO_API_KEY')

In [125]:
def extract_answer(transcript: str) -> str:

    prompt_template = PromptTemplate(
            template=hard_luck_prompt,
            input_variables=["transcript"],
        )
    formatted_input = prompt_template.format_prompt(transcript=transcript)
    
    # llm = ChatOpenAI(model="gpt-3.5-turbo-0125")
    # llm = ChatOpenAI(model="gpt-4-0125-preview")
    llm = ChatAnthropic(model="claude-instant-1.2")
    # llm = ChatAnthropic(model="claude-2.1")

    # print( formatted_input.to_string())
    print(formatted_input.to_string())
    # answer = llm.invoke(formatted_input.to_string())

    answer = llm.generate_content(formatted_input.to_string()).text
    
    return answer

In [126]:
# client = pymongo.MongoClient(os.getenv('mongodb_uri'))
# DB_NAME = "hibt_transcripts"
# COLLECTION_NAME = "hibt_test_openai_collection"
# ATLAS_VECTOR_SEARCH_INDEX_NAME = "openai_vector_index"
# MONGODB_COLLECTION = client[DB_NAME][COLLECTION_NAME]

# MONGODB_ANSWER_COLLECTION = client[DB_NAME]["hibt_test_openai_answer_collection"]

In [127]:
# # get the max trans_chunk_num where company = company in the collection
# company = "ActOne Group"

# max_trans_chunk_num = MONGODB_COLLECTION.find_one({"company": company}, sort=[("trans_chunk_num", -1)])["trans_chunk_num"]

# # in the MONGODB_COLLECTION, get the last 3 documents where company = company sorted by trans_chunk_num in descending order
# transcripts = MONGODB_COLLECTION.find({"company": company}).sort("trans_chunk_num", -1).limit(3)

# for doc in transcripts:
#     print(doc['transcript'])

In [128]:
# Find all unique values for the 'company' field
unique_companies = MONGODB_COLLECTION.distinct('company')

embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

# Create a list to store the extracted answers
extracted_answers = []

# Print all unique companies
for company in unique_companies[:10]:
    query = "how much of your success do you attribute to how hard you worked, skill, and/or your intelligence and how much do you think it had to do with luck?"

    # 

    results = MONGODB_COLLECTION.aggregate([
        {"$vectorSearch": {
            "index": ATLAS_VECTOR_SEARCH_INDEX_NAME,
            "path": "embedding",
            "queryVector": embeddings.embed_query(hard_luck_prompt),
            "numCandidates": 100,
            "limit": 3,
            "filter": {"company": company}
        }}
    ])
    print(company)
    # if results is empty then continue
    if results:
        transcript = ""
        for i, doc in enumerate(results):
            chunk_num = doc['trans_chunk_num']
            org_filename = doc['org_filename']
            guest = doc['guest']
            transcript += doc['transcript']
            try:
                next_chunk = MONGODB_COLLECTION.find_one({"org_filename": org_filename, "trans_chunk_num": chunk_num + 1})
                transcript += next_chunk['transcript']
            except:
                continue
        extracted_answer = extract_answer(transcript)

        answer = {
            "company": company,
            "guest": guest,
            "org_filename": org_filename,
            "answer": extracted_answer
        }
        extracted_answers.append(answer)
        print(company, guest, "\n", extracted_answer, "\n")
        print("---------------------------------------------------")
    else:
        print("No results found for", company)

AMP Robotics

    You are a transcription analyst assistant. You are given portions of a transcription in order to extract the guests answer.
    We are working with transcriptions from the podcast "How I Built This" where the host Guy Raz interviews entrepreneurs. 
    Typically towards the end of the interview he will ask a question regarding to what extent the entrepreneur attributes their success to skill, hard work, and/or intelligence versus luck.
    Your task is to extract the answer from the transcriptions. Remember that these are transcriptions so words can be mispelled like "lock" instead of "luck" or "intelligents" instead of "intelligence".

    The answer is typically a few sentences long and can be found in the same paragraph or in the following paragraphs depending on their thoughts.

    <Examples>
    
    Example Question: When you think about the success of what you created, how much do you think it has to do with, with how hard you worked and your intelligence and 

In [111]:
import json

# Specify the file path where you want to save the data
file_path = "../answers/Anthropic_ClaudeInstant1_2_embedprompt.json"

# Save the extracted_answers list as JSON with indentation
with open(file_path, "w") as file:
    json.dump(extracted_answers, file, indent=4)


In [3]:
# check how many documents are in MONGODB_ANSWER_COLLECTION
MONGODB_ANSWER_COLLECTION.count_documents({})

149

In [3]:
# check how many documents in MONGODB_ANSWER_COLLECTION has the answer "Question/Answer not found."
MONGODB_ANSWER_COLLECTION.count_documents({"answer": "Question/Answer not found."})

32

In [13]:
# get every unique "org_filename" in the collection
all_files = MONGODB_COLLECTION.distinct('org_filename')
# place all the unique "org_filename" in a list
all_files_list = list(all_files)

In [16]:
# get a list of all of the files in hibt_main
path = '../podscribe_transcription/hibt_main/'
all_trans = os.listdir(path)
# place all the unique "org_filename


In [19]:
len(all_trans)

257

In [20]:
len(all_files_list)

243

In [18]:
# display all files that are in all_trans but not in all_files_list
missing_files = [file for file in all_trans if file not in all_files_list]
missing_files

['Nov.02 2021, Hear Don Katz on the Amazing Story of Audible’s Founding-transcript.txt',
 'Tofurky_ Seth Tibbott (2019)-transcript.txt',
 'Remembering Jake Burton Carpenter-transcript.txt',
 'Discord_ Jason Citron-transcript.txt',
 'Walker & Company_ Tristan Walker-transcript.txt',
 "Angie's BOOMCHICKAPOP_ Angie & Dan Bastian (2019)-transcript.txt",
 'Milk Bar_ Christina Tosi-transcript.txt',
 "Luke's Lobster_ Luke Holden and Ben Conniff-transcript.txt",
 'OtterBox_ Curt Richardson-transcript.txt',
 'McBride Sisters Wine (Part 1 of 2)_ Robin McBride and Andréa McBride John-transcript.txt',
 'Spikeball_ Chris Ruder-transcript.txt',
 "Raising Cane's_ Todd Graves-transcript.txt",
 'Leatherman Tool Group_ Tim Leatherman-transcript.txt',
 'Sierra Nevada Brewing Company_ Ken Grossman-transcript.txt',
 'Tapping the heat beneath your feet with Kathy Hannun of Dandelion Energy-transcript.txt',
 'DoorDash_ Tony Xu-transcript.txt',
 'Twitch_ Emmett Shear-transcript.txt',
 'When your dinner is pri

In [4]:
#get the list of companies that have the answer "Question/Answer not found."
companies_with_no_answer = MONGODB_ANSWER_COLLECTION.find({"answer": "Question/Answer not found."})
# put them into a list and print them
companies_with_no_answer_list = []
for company in companies_with_no_answer:
    companies_with_no_answer_list.append(company['company'])

257

In [9]:
# find all of the files that include the companies in companies_with_no_answer_list
files_with_no_answer = []
for file in all_files:
    for company in companies_with_no_answer_list:
        if company in file:
            print(file)
            print(company)
            files_with_no_answer.append(file)

Literally unearthing a climate solution with Cody Finke of Brimstone-transcript.txt
Brimstone
A biometric smart gun with Kai Kloepfer of Biofire-transcript.txt
Biofire
Cultivating a creative community with Tina Roth-Eisenberg of CreativeMornings-transcript.txt
CreativeMornings
Reclaiming food waste with Jasmine Crowe-Houston of Goodr (2022)-transcript.txt
Goodr (2022)
Pitchfork_ Ryan Schreiber-transcript.txt
Pitchfork
Live From HIBT Summit_ Payal Kadakia, Tristan Walker, and Perry Chen on Innovation-transcript.txt
Live From HIBT Summit
When trucks drive themselves with Chris Urmson of Aurora-transcript.txt
Aurora
Threading the future of circular fashion with Peter Majeranowski of Circ-transcript.txt
Circ
Chipotle_ Steve Ells (2017)-transcript.txt
Chipotle
When a robot cooks your lunch with Steve Ells of Chipotle and Kernel-transcript.txt
Chipotle
When a robot cooks your lunch with Steve Ells of Chipotle and Kernel-transcript.txt
Chipotle and Kernel
Shooting for the moon with Steve Alte

In [72]:
# I need to empty this mongodb collection MONGODB_COLLECTION
MONGODB_ANSWER_COLLECTION.delete_many({})

DeleteResult({'n': 236, 'electionId': ObjectId('7fffffff0000000000000117'), 'opTime': {'ts': Timestamp(1708139348, 68), 't': 279}, 'ok': 1.0, '$clusterTime': {'clusterTime': Timestamp(1708139348, 68), 'signature': {'hash': b')i#}G\x8a\xc9K\xb9\xcc$n2\xc3Z\x9a\xf9\xc3|\xac', 'keyId': 7297236172768542722}}, 'operationTime': Timestamp(1708139348, 68)}, acknowledged=True)

In [8]:
client = pymongo.MongoClient(os.getenv('mongodb_uri'))
DB_NAME = "hibt_transcripts"
COLLECTION_NAME = "hibt_test_collection"
ATLAS_VECTOR_SEARCH_INDEX_NAME = "hibt_test_index"
MONGODB_COLLECTION = client[DB_NAME][COLLECTION_NAME]

In [9]:
# MONGODB_COLLECTION.delete_many({"$expr": {"$ne": [{"$size": "$embedding"}, 768]}})


DeleteResult({'n': 20, 'electionId': ObjectId('7fffffff0000000000000116'), 'opTime': {'ts': Timestamp(1707905272, 26), 't': 278}, 'ok': 1.0, '$clusterTime': {'clusterTime': Timestamp(1707905272, 26), 'signature': {'hash': b'\x9c\x92cT\x081\xf2\xc03M\xa1\xdc\xe8\xdd\x82\xf6\xa7\x8aX\t', 'keyId': 7297236172768542722}}, 'operationTime': Timestamp(1707905272, 26)}, acknowledged=True)

In [21]:
import nltk
nltk.download('punkt')  # Download the tokenizer model if not already downloaded

# Function to count tokens in a text file
def count_tokens(filename):
    # Open and read the file
    with open(filename, 'r', encoding='utf-8') as file:
        text = file.read()
    
    # Tokenize the text
    tokens = nltk.word_tokenize(text)
    
    # Return the number of tokens
    return len(tokens)



[nltk_data] Downloading package punkt to /Users/leowalker/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [22]:

# Example usage
filename = '../podscribe_transcription/hibt_main/Coinbase_ Brian Armstrong-transcript.txt'  # Replace this with your text file's path
num_tokens = count_tokens(filename)
print(f"The text file contains {num_tokens} tokens.")

The text file contains 22102 tokens.
